In [1]:
import os
import glob
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
root = '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/exps/monitor/'

In [4]:
def smooth(vals,weight=0.6):
  smoothed = []
  last = vals[0]
  for point in vals:
    smoothed_val = last * weight + (1 - weight) * point
    smoothed.append(smoothed_val)
    last = smoothed_val
  return smoothed

In [5]:
def num_columns(header):
    for i in range(len(header)):
        if header[i].startswith('Click_') or header[i].startswith('Dur_') or header[i].startswith('base_'):
            return i
        
start_hour = 1e32
end_hour = -1

def gen_dfs(root):
  global start_hour, end_hour
  dfs = []
  for dir in glob.glob('%s/*' % root):
    if os.path.isdir(dir):
        name = os.path.basename(dir)
#         print(name)
        metric_file = os.path.join(dir, 'metric_hours.txt')
        if os.path.exists(metric_file):
            header = open(metric_file).readline().strip().split('\t')
            columns = num_columns(header)
            l = []
            for i, line in enumerate(open(metric_file)):
                if i == 0:
                    continue
                vals = line.strip().split('\t')
                if not vals:
                    continue
                if len(vals) < columns:
                    continue
                vals = vals[:columns]
                vals = [float(x) for x in vals]
                vals[0] = int(vals[0])
                hour = vals[0]
                if hour > end_hour:
                  end_hour = hour
                if hour < start_hour:
                  start_hour = hour
                l.append(vals)
            df = pd.DataFrame.from_dict(l)
            df.columns = header[:columns]

#             df = pd.read_csv(metric_file, sep='\t', error_bad_lines=False)
#             df = df[df.base_group_click_time_auc < 1.]
#             if 'day' in name:
            df = df.groupby(df.hour, as_index=False).last()
            df = df.sort_values(by=['hour'])
            df.name = name
            dfs.append(df)
  return dfs

In [6]:
dfs = gen_dfs(root)
# def get_base(df):
#     base = df[df.base_group_click_time_auc < 1.].reset_index()
#     for key in base.columns:
#         if key.startswith('base_'):
#             base[key.replace('base_', '')] = base[key]
#     return base

# base = pd.concat([get_base(x) for x in dfs])
# # base = base.sort_values('gold_auc').groupby(['hour']).tail(1)
# ## TODO how to find max gold/auc row after groupby
# base = base.groupby(base.hour).last()
# # base = base.groupby(base.hour).agg({'gold_auc':'max'})
# # idx = base.groupby(['hour'])['gold_auc'].transform(max) == base['gold_auc']
# # base = base.sort_values(by=['hour'])
# # base = base[idx]

# base.name = 'base'
# dfs.append(base)
metric_file = '/home/gezi/tmp/rank/log/score_info.15.txt'
input = [x.strip() for x in open(metric_file).readlines()]
input.reverse()

data = []
is_online = False
testid = None
testid_ = None
tags = set()
for line in input: 
  try:
    l = line.split('\t')
    if len(l) != 4:
      continue
    
    name, hour, mark, info = l
    
    if 'inverse_ratio' in name:
      continue
    
    tag = '\t'.join(l[:3])
    
    if tag in tags:
      continue 
    else:
      tags.add(tag)
    
    if ',' in name:
      testid = 'base'
    else:
      testid = None

    if 'online' in name:
      is_online = True
    else:
      is_online = False
    
    if not testid:
      testid = name.split('abid')[-1]
      testid_ = testid
    l = []
    for x in info.split():
        if x.startswith('version'):
            continue
        key, val = x.split(':')
        key = key.replace('/', '_')
        l.append((key, float(val)))
#     print(l)
    infos = dict(l)
    infos['hour'] = int(hour)
    infos['mark'] = mark
    infos['abtest'] = testid
    infos['online'] = is_online
    data.append(infos)
  except Exception:
    # print(line)
    pass
# print(len(data))
df2 = pd.DataFrame(data)
df2 = df2.sort_values(by=['hour'])

df2 = df2[df2.hour >= start_hour]
df2 = df2[df2.hour <= end_hour]
df2 = df2[df2.abtest=='base']

df2 = df2[df2.mark=='tuwen']
df2 = df2[df2.online==False]

base = df2
base.name = 'base'

if len(base) > 0:
  dfs.append(base)

m = {}
for df in dfs:
    m[df.name] = df
m.keys()

for df in dfs:
    df['mean_auc'] = (df['auc'] + df['click_time_auc']) / 2
    df['group_mean_auc'] =  (df['group_auc'] + df['group_click_time_auc']) / 2
    df['wmean_auc'] = (df['auc'] + df['click_weighted_time_auc']) / 2
    df['group_wmean_auc'] =  (df['group_auc'] + df['group_click_weighted_time_auc']) / 2

In [7]:
# dfs = [x for x in dfs if 'smooth' not in x.name]
dfs[0]

,hour,loss,auc,time_auc,weighted_time_auc,pos_ratio,click_time_auc,click_weighted_time_auc,group_auc,group_pos_ratio,...,group_click_top3_click_rate,gold_auc,click_ratio,time_per_show,time_per_click,time_per_user,mean_auc,group_mean_auc,wmean_auc,group_wmean_auc
0,2019112923,0.76065,0.71617,0.70443,0.74213,0.14509,0.56434,0.62498,0.65443,0.16134,...,1.0,0.61039,0.14509,10.96485,75.57522,206.40442,0.640255,0.610385,0.670575,0.675015
1,2019113000,0.72649,0.60676,0.60130,0.62524,0.13975,0.53261,0.56253,0.54451,0.15825,...,1.0,0.53796,0.13975,10.08804,72.18676,180.11396,0.569685,0.537960,0.584645,0.602070
2,2019113001,0.74865,0.71548,0.70393,0.74421,0.14216,0.56232,0.59472,0.65026,0.16315,...,1.0,0.60701,0.14216,10.34046,72.74017,180.57128,0.638900,0.607015,0.655100,0.675680
3,2019113002,0.76100,0.74094,0.73039,0.76858,0.13495,0.59287,0.59851,0.66519,0.15954,...,1.0,0.61448,0.13495,9.43799,69.93888,169.34605,0.666905,0.614485,0.669725,0.683530
4,2019113003,0.75260,0.75123,0.73967,0.77443,0.13758,0.59209,0.57781,0.67345,0.16559,...,1.0,0.61378,0.13758,9.97024,72.46719,167.14134,0.671660,0.613775,0.664520,0.682455
5,2019113004,0.75502,0.75477,0.74504,0.79314,0.13463,0.61741,0.60623,0.67600,0.16677,...,1.0,0.61800,0.13758,9.97024,72.46719,167.14134,0.686090,0.618000,0.680500,0.682450
6,2019113005,0.75545,0.75400,0.74260,0.78510,0.14426,0.60525,0.61210,0.67017,0.17363,...,1.0,0.61726,0.13758,9.97024,72.46719,167.14134,0.679625,0.617260,0.683050,0.683865
7,2019113006,0.75940,0.73320,0.72093,0.75899,0.15338,0.58382,0.61560,0.67751,0.17490,...,1.0,0.62773,0.13758,9.97024,72.46719,167.14134,0.658510,0.627730,0.674400,0.696060
8,2019113007,0.76180,0.73181,0.71984,0.75779,0.14911,0.58167,0.62526,0.67812,0.16835,...,1.0,0.62299,0.13758,9.97024,72.46719,167.14134,0.656740,0.622990,0.678535,0.689400
9,2019113008,0.76062,0.73340,0.72076,0.75001,0.14966,0.57527,0.60040,0.67097,0.16792,...,1.0,0.62131,0.13758,9.97024,72.46719,167.14134,0.654335,0.621310,0.666900,0.686380


In [8]:
def show(dfs, names, smoothing=0., include=None, base=True,  limit=0):
  if isinstance(names, str):
    names = names.split(',')
  ori_names = names
  names = [x.replace('/', '_') for x in names]

# #   fig.update_layout(height=600, width=800, title_text="Subplots")
# #   fig.show()
  
  for name, name2 in zip(names, ori_names):
    datas = []
    for df in dfs:
        if include and include not in df.name and not df.name == 'base':
            continue
        hours=df.hour.values if not limit else df.hour.values[-limit:]
        mode = 'lines+markers' if not 'day' in df.name else 'markers'
        mode = 'lines+markers'
        if 'lines' in mode and smoothing > 0:
            mode = mode.replace('+markers', '')
        if include == 'day':
            mode = 'lines+markers'
        data = go.Scatter(
            x=[datetime.strptime(str(x), '%Y%m%d%H') for x in hours],
            y=smooth(df[name].values if not limit else df[name].values[-limit:], smoothing),
            mode=mode,
            line_shape='spline',
            line_smoothing=0.5,
            marker = dict(size=4),
            name=df.name,
        )

        datas.append(data)
#     if base:
# #         dfs2 = [dfs[0], dfs[2]]
# #         for i in range(2):
#         data = go.Scatter(
#             x=[datetime.strptime(str(x), '%Y%m%d%H') for x in dfs[0].hour],
#             y=smooth(dfs[0]['base_' + name].values, smoothing),
#             mode='markers+lines',
#             line_shape='spline',
#             line_smoothing=0.5,
#             marker = dict(size=4),
#             name='base'
#         )
#         datas.append(data)

#     for data in datas:
#         fig.add_trace(data)
    layout = go.Layout(xaxis=dict(type='date'), title=name2, hovermode='x')
    fig = go.Figure(data=datas, layout=layout)
    py.iplot(fig)
#   rows = int(len(names) / 3 + 0.5)
#   cols = 3
#   if len(names) < cols:
#     rows = 1
#     cols = len(names)
#   fig = make_subplots(rows=rows, cols=cols)
#   fig.show()

In [9]:
#metric_names = 'gold_auc,group_auc,group/click/time_auc,auc,click/time_auc,click/weighted_time_auc,time_auc,weighted_time_auc'
metric_names = 'gold/auc,group/auc,group/click/time_auc,auc'
metric_names = [
    'gold/auc',
]
# show(dfs, metric_names, base=False, smoothing=0.)
dfs_ = dfs
models = m.keys()
dfs_ = [m[key] for key in models]
# show(dfs_, metric_names)
show(dfs_, metric_names, smoothing=0.5)

In [10]:
metric_names = [
 'group/mean/auc',
 'group/wmean/auc',
 'group/auc',
 'group/click/time_auc',
 'group/click/weighted_time_auc',
 'mean/auc',
 'wmean/auc',
 'auc',  
 'click/time_auc',
 'click/weighted_time_auc',
]
models = [
#     'base.days',
#     'time.days',
#     'base',
#     'base.hours',
#     'time.hours',
#     'time.smooth.hours',
#     'time.4bins.hours',
#     'time.taskmlp.hours',
#     'time.smooth.taskmlp.hours',
#     'time.4bins.3shift.taskmlp.hours',
#     'time.4bins.taskmlp.hours',
#     'taskmlp.hours',
#     'time.smooth.taskmlp',
] 

if not models:
    models = m.keys()

# models += [key for key in m if key.startswith('time.')]
dfs_ = [m[key] for key in set(models) if key in m]
show(dfs_, metric_names, smoothing=0.)

In [11]:
show(dfs_, metric_names, smoothing=0.5)

In [12]:
metric_names = [
 'group_auc',
 'group/click/time_auc',
 'auc',   
]
show(dfs_, metric_names, smoothing=0.)

In [13]:
# metric_names = 'gold_auc,group_auc,group/click/time_auc'
# show(dfs, metric_names, base=True, include='day')

In [14]:
# show(dfs, 'gold_auc,group_auc,group_click_time_auc,group_click_weighted_time_auc,auc,weighted_time_auc,click_time_auc,click_weighted_time_auc', base=True)